In [ ]:
import cv2
import os
from keras.models import load_model
!pip install tqdm
from tqdm import tqdm
import json
import numpy as np
from glob import glob

In [ ]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import load_model

# Ensure the haarcascade file is in the correct path
cascPath = os.path.abspath(os.path.dirname(os.path.dirname(os.getcwd()))) + "/haarcascade_frontalface_alt.xml"
# Check if the file exists
if not os.path.isfile(cascPath):
    raise FileNotFoundError(f"File not found: {cascPath}")
# Load the Haar Cascade classifier
faceCascade = cv2.CascadeClassifier(cascPath)
# Load the pre-trained emotion detection model
model = load_model("emotion_model.keras")

# Function to process video and save frames with predictions
def runGetImage(video_path, output_path):
    # capture video
    video_capture = cv2.VideoCapture(video_path)
    # get total frames
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # check if video opened successfully
    if not video_capture.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    # create empty list to store frames
    frames = []

    # loop through video frames
    for _ in tqdm(range(total_frames), desc="Processing frames"):
        # read frame from video
        ret, frame = video_capture.read()
        # check if frame is read correctly
        if not ret:
            break

        # convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect faces in the frame
        faces = faceCascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE
        )

        # check if any faces are detected
        if len(faces) > 0:
            # take the largest face detected
            x, y, w, h = max(faces, key=lambda face: face[2] * face[3])
            # draw rectangle around the face 
            face = gray[y:y+h, x:x+w]
            # resize the face to 48x48 pixels
            face_resized = cv2.resize(face, (48, 48))
            # normalize the pixel values to be between 0 and 1
            face_input = face_resized.reshape(1, 48, 48, 1).astype('float32') / 255.0

            # predict the emotion using the model
            result = model.predict(face_input, verbose=0)  # shape (1, N)
            # get the index of the class with the highest probability
            frames.append(result[0].tolist())
        else:
            # if no face is detected, append an empty list
            frames.append([])

    # release the video capture object
    video_capture.release()

    # write the frames to a JSON file
    # create the output directory if it doesn't exist
    with open(output_path, "w") as f:
        json.dump(frames, f, indent=2)

    print(f"Finished processing {video_path}. Saved output to {output_path}")


In [ ]:
# File paths
video_paths = "<path_to_video_files>"
keypoints_path = "<path_to_keypoints_files>"
output = "<path_to_output_json>"

# read video files and keypoints files
keypoints_files = glob(os.path.join(keypoints_path, "*_keypoints.pth"))
existing_files = glob(os.path.join(output, "*.json"))

counter = 0
skip = False

# list all video files in the directory
for v in os.listdir(video_paths):
    # list all video files in the existing files list
    for e in existing_files:
        # build the base name of the existing file
        base_name = os.path.basename(e).replace(".json", "")
        # check if the video file name matches the existing file name
        if (v[:-4] == base_name):
            # increment the counter and print the message
            counter += 1
            print("Found existing file for video: ", v, " counter = ", counter)
            skip = True
            break
    # if skip is True, continue to the next iteration
    if skip:
        skip = False
        continue
    # if skip is False, check for keypoints files
    for k in keypoints_files:
        # build the base name of the keypoints file
        base_name = os.path.basename(k).replace("_keypoints.pth", "")
        # check if the keypoints file name matches the video file name
        if (v[:-4] == base_name):
            # increment the counter and print the message
            counter += 1
            print("Found keypoints file for video: ", v, " counter = ", counter)
            # call the runGetImage function to process the video and save the frames with predictions
            runGetImage(os.path.join(video_paths,v), os.path.join(output, v[:-4] +".json"))
            break

Found existing file for video:  5086465773912997411.mp4  counter =  1
Found existing file for video:  5087953980081062580.mp4  counter =  2
Found existing file for video:  5090546422340930233.mp4  counter =  3
Found existing file for video:  5090916219025116054.mp4  counter =  4
Found existing file for video:  5092765202446045704.mp4  counter =  5
Found existing file for video:  5093160768934007504.mp4  counter =  6
Found existing file for video:  5093521546186871869.mp4  counter =  7
Found existing file for video:  5093895208341624036.mp4  counter =  8
Found existing file for video:  5094257274914886988.mp4  counter =  9
Found existing file for video:  5097975856769557081.mp4  counter =  10
Found existing file for video:  5099091689273058848.mp4  counter =  11
Found existing file for video:  5099449889545545450.mp4  counter =  12
Found existing file for video:  5101325932090649776.mp4  counter =  13
Found existing file for video:  5103182645622502287.mp4  counter =  14
Found keypoints

Processing frames: 100%|██████████| 89464/89464 [1:44:25<00:00, 14.28it/s]  

Released video file: D:\original_data\5104636062555440832.mp4
Saved results to C:\Users\taylo\OneDrive\Documents\Dissertation_Work\BOBSL\FED_output_data\5104636062555440832.json (JSON Lines format)
Found existing file for video:  5105777664862718183.mp4  counter =  16
Found existing file for video:  5106137153625393471.mp4  counter =  17
Found existing file for video:  5108723153434317714.mp4  counter =  18
Found existing file for video:  5113588492387230304.mp4  counter =  19
Found existing file for video:  5120215198258444599.mp4  counter =  20
Found existing file for video:  5125058632047937012.mp4  counter =  21
Found existing file for video:  5129558040617440365.mp4  counter =  22
